<a href="https://www.kaggle.com/code/raviiloveyou/project-pii?scriptVersionId=165491333" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json


In [2]:
!pip install seqeval evaluate -q

In [3]:
import json
import argparse
from itertools import chain
from functools import partial
#import warnings
#warnings.filter("ignore")
import torch
from transformers import AutoTokenizer,Trainer,TrainingArguments
from transformers import AutoModelForTokenClassification,DataCollatorForTokenClassification
import evaluate
from datasets import Dataset,features
import numpy as np

2024-03-05 04:46:05.313308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 04:46:05.313519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 04:46:05.463433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
data=json.load(open("/kaggle/input/pii-detection-removal-from-educational-data/train.json"))


In [5]:
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
]

print(id2label)

{0: 'B-EMAIL', 1: 'B-ID_NUM', 2: 'B-NAME_STUDENT', 3: 'B-PHONE_NUM', 4: 'B-STREET_ADDRESS', 5: 'B-URL_PERSONAL', 6: 'B-USERNAME', 7: 'I-ID_NUM', 8: 'I-NAME_STUDENT', 9: 'I-PHONE_NUM', 10: 'I-STREET_ADDRESS', 11: 'I-URL_PERSONAL', 12: 'O'}


In [6]:
def tokenize(example,tokenizer,label2id,max_length):
    text=[]
    labels=[]
    for t,l  in zip(
        example["tokens"],example["provided_labels"]):
        text.append(t)
        labels.extend([l]*len(t))
        ws=True
        if ws:
            text.append(" ")
            labels.append("O")
            
        tokenized=tokenizer("".join(text),return_offsets_mapping=True,max_length=max_length)
        labels=np.array(labels)
        text="".join(text)
        token_labels=[]
        for idx,end_idx in tokenized.offset_mapping:
            if idx==0 and end_idx==0:
                token_labels.append(label2id["O"])
                continue
            if idx < len(text) and end_idx <= len(text):  # Check if the index is within bounds
                if text[idx].isspace():
                    idx += 1
                token_labels.append(label2id[labels[idx]])
            else:
                token_labels.append(label2id["O"])

        length=len(tokenized.input_ids)
        return {**tokenized,"labels":token_labels,"length":length}

In [7]:
tokenizer=AutoTokenizer.from_pretrained("roberta-base")
ds=Dataset.from_dict({
    "full_text":[x["full_text"]for x in data],
    "document":[str(x["document"])for x in data],
    "tokens":[x["tokens"]for x in data],
    #"training_whitespace":[x["trailing_whitespace"]for x in data],
    "provided_labels":[x["labels"]for x in data]
})
ds=ds.map(tokenize,fn_kwargs={"tokenizer":tokenizer,"label2id":label2id,"max_length":1024},num_proc=5)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


#3:   0%|          | 0/1361 [00:00<?, ?ex/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


#4:   0%|          | 0/1361 [00:00<?, ?ex/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/datasets/table.py:1291: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/opt/conda/lib/python3.10/site-packages/datasets/table.py:1317: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [8]:
x = ds[1:5]
for r,g in zip(x["tokens"], x["provided_labels"]):
    if g != "O":
        print((r,g))
print("*" * 100)

for ids, labels in zip(x["input_ids"], x["labels"]):
    for id, label in zip(ids, labels):
        if id2label[label] != "O":
            token = tokenizer.convert_ids_to_tokens(id)
            print((token, id2label[label]))

print("*" * 100)



(['Diego', 'Estrada', '\n\n', 'Design', 'Thinking', 'Assignment', '\n\n', 'Visualization', 'Tool', '\n\n', 'Challenge', '&', 'Selection', '\n\n', 'The', 'elderly', 'were', 'having', 'a', 'hard', 'time', 'adapting', 'to', 'the', 'changes', 'we', 'brought', 'in', 'our', 'bank', '.', 'As', ' ', 'a', 'result', 'of', 'a', 'poorly', 'implemented', 'linear', 'solution', ',', 'a', 'more', 'customer', 'centric', 'approach', 'was', ' ', 'needed', '.', '\n\n', 'After', 'learning', 'about', 'design', 'thinking', 'in', 'this', 'course', ',', 'we', 'decided', 'to', 'apply', 'it', 'to', 'solve', 'this', ' ', 'problem', '.', 'The', 'visualization', 'tool', 'allowed', 'the', 'team', 'to', 'create', 'a', 'dynamic', 'presentation', 'using', ' ', 'diagrams', ',', 'figures', 'and', 'drawings', 'on', 'the', 'go', 'that', 'really', 'resonated', 'among', 'the', 'stakeholders', '.', ' ', 'Previous', 'to', 'this', 'change', ',', 'none', 'of', 'our', 'solutions', 'seemed', 'to', 'be', 'adequate', 'for', 'them', 

In [9]:
from seqeval.metrics import recall_score,precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

In [10]:
def compute_metrics(p,all_labels):
    predictions,labels=p
    predictions=np.argmax(predictions,axis=2)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

In [11]:
model =AutoModelForTokenClassification.from_pretrained(
    "roberta-base",
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True)
collator=DataCollatorForTokenClassification(tokenizer,pad_to_multiple_of=16)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
args = TrainingArguments(
    output_dir="output", 
    fp16=False,
    learning_rate=1e-5,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=3,
    report_to="none",
    evaluation_strategy="no",
    do_eval=False,
    save_total_limit=1,
    logging_steps=20,
    lr_scheduler_type='cosine',
    metric_for_best_model="f1",
    greater_is_better=True,
    warmup_ratio=0.1,
    weight_decay=0.01
)

In [13]:
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=ds,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics,all_labels=all_labels))

In [14]:
%%time
trainer.train()

Step,Training Loss
20,2.506500
40,2.211400
60,1.186400
80,0.145700
100,0.125800
120,0.065800
140,0.051600
160,0.037300
180,0.020200
200,0.010900


CPU times: user 2h 11min 27s, sys: 13min 23s, total: 2h 24min 50s
Wall time: 1h 13min 26s


TrainOutput(global_step=1415, training_loss=0.09662286286232434, metrics={'train_runtime': 4405.4028, 'train_samples_per_second': 7.726, 'train_steps_per_second': 0.321, 'total_flos': 277622748865152.0, 'train_loss': 0.09662286286232434, 'epoch': 4.99})

In [15]:
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#secret_value_0 = user_secrets.get_secret("kaggle")

#import wandb
#wandb.login(key=secret_value_0)
#wandb.init(project="PII",job_type="preprocessing_pipelines!")

In [16]:
#wandb.config.update({
 #   "max_length":max_length,
  #  "doc_stride":doc_stride
#})

In [17]:
#train_data.to_parquet("new_data.parquet",index=False)
#stride_df.to_parquet("stride_data.parquet",index=False)

#new_data=wandb.Artifact(name="new_data",type="dataset")
#new_data.add_file("new_data.parquet")
#wandb.log_artifact(new_data)

#processed_data=wandb.Artifact(name="processed_data",type="dataset")
#processed_data.add_file("stride_data.parquet")
#wandb.log_artifact(processed_data)

In [18]:
#wandb_htmls=[wandb.Html(visualize(row)) for _, row in train_data.iterrows()]
#train_data["visualization"]=wandb_htmls
#table=wandb.Table(dataframe=train_data)
#wandb.log({
 #   "original_data":table
#})

In [19]:
#wandb.finish()